In [6]:
import numpy as np
import pandas as pd

import tensorflow as tf
from sklearn.base import TransformerMixin
from sklearn.preprocessing import Imputer, LabelBinarizer, OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline, make_union
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import metrics
from keras.utils import to_categorical

from sklearn.datasets import load_breast_cancer, load_boston, load_iris

import matplotlib.pyplot as plt
%matplotlib inline

In [78]:
####### Mandatory tensorflow setup
# 1) reset graph
# 2) setup tf variables
# 3) init
# 4) "with tf.Session() as sess:"

tf.reset_default_graph()

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = tf.multiply(x, y)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    results = f.eval()
    print(results)

12


## Regression

In [79]:
data = load_boston()

### MUST reshape target (y) to be list of lists for tensorflow 
target = data.target.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(data.data, target)

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [80]:
tf.reset_default_graph()

X = tf.placeholder(dtype=tf.float32, shape=(None, X_train.shape[1]), name='X')
y = tf.placeholder(dtype=tf.float32, shape=(None), name='y')

# first hidden layer
# X == features coming in, 13 == number of features == neurons
h1 = tf.layers.dense(X, 13, name='hidden1', activation=tf.nn.relu)

# last/output always with 1 neuron for regression problem
y_hat = tf.layers.dense(h1, 1, name='y_hat', activation=None)

loss = tf.losses.mean_squared_error(y, y_hat)

gd = tf.train.AdamOptimizer(.1) #could pass in learning rate as param
training_op = gd.minimize(loss)

saver = tf.train.Saver()

In [81]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    #train model
    for epoch in range(100):
        sess.run(training_op, feed_dict={X: X_train, y: y_train})
        if (epoch % 10 == 0):
            training_loss = sess.run(loss, feed_dict={X: X_train, y: y_train})
            test_loss = sess.run(loss, feed_dict={X: X_test, y: y_test})
            print('epoch:',epoch,' | training loss:', training_loss, ' | test loss:', test_loss)
            
    saver.save(sess, './boston.ckpt')
    pred = sess.run(y_hat, feed_dict={X: X_test})

epoch: 0  | training loss: 498.82523  | test loss: 578.54834
epoch: 10  | training loss: 122.30233  | test loss: 123.692345
epoch: 20  | training loss: 42.74601  | test loss: 45.783573
epoch: 30  | training loss: 22.761417  | test loss: 32.743988
epoch: 40  | training loss: 16.274607  | test loss: 22.88149
epoch: 50  | training loss: 12.187437  | test loss: 18.324211
epoch: 60  | training loss: 10.561711  | test loss: 19.790585
epoch: 70  | training loss: 9.381224  | test loss: 16.903927
epoch: 80  | training loss: 8.732194  | test loss: 16.200596
epoch: 90  | training loss: 8.143583  | test loss: 16.18833


In [82]:
with tf.Session() as sess:
    saver.restore(sess, 'boston.ckpt')
    pred = sess.run(y_hat, feed_dict={X: X_test})
    
metrics.r2_score(y_test, pred)

INFO:tensorflow:Restoring parameters from boston.ckpt


0.8251895825475513

## Binary Classification

In [83]:
data = load_breast_cancer()
target = 1 - data.target

### MUST reshape target (y) to be list of lists for tensorflow 
target = data.target.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(data.data, target)

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [84]:
tf.reset_default_graph()

#None == allow dynamic number of rows; X_train.shape[1] == number of features
X = tf.placeholder(dtype=tf.float32, shape=(None, X_train.shape[1]), name='X')
y = tf.placeholder(dtype=tf.float32, shape=(None), name='y')

# first hidden layer
# X == features coming in, 30 == number of features == neurons
h1 = tf.layers.dense(X, 30, name='hidden1', activation=tf.nn.relu)

# second hidden layer
# X == features coming in, 26 == number of neurons == arbitrary
h2 = tf.layers.dense(h1, 26, name='hidden2', activation=tf.nn.relu)

# last/output always with 1 neuron for binary classification problem
y_hat = tf.layers.dense(h2, 1, name='y_hat', activation=tf.nn.sigmoid)

loss = tf.losses.log_loss(y, y_hat)

gd = tf.train.AdamOptimizer(.01) #could pass in learning rate as param
training_op = gd.minimize(loss)

saver = tf.train.Saver()

In [85]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    #train model
    for epoch in range(100):
        sess.run(training_op, feed_dict={X: X_train, y: y_train})
        if (epoch % 10 == 0):
            training_loss = sess.run(loss, feed_dict={X: X_train, y: y_train})
            test_loss = sess.run(loss, feed_dict={X: X_test, y: y_test})
            print('epoch:',epoch,' | training loss:', training_loss, ' | test loss:', test_loss)
            
    saver.save(sess, './breast_cancer.ckpt')
    pred = sess.run(y_hat, feed_dict={X: X_test})

epoch: 0  | training loss: 0.6027077  | test loss: 0.5729999
epoch: 10  | training loss: 0.11087423  | test loss: 0.11399212
epoch: 20  | training loss: 0.059190013  | test loss: 0.06608006
epoch: 30  | training loss: 0.04484703  | test loss: 0.055342205
epoch: 40  | training loss: 0.032626305  | test loss: 0.054871682
epoch: 50  | training loss: 0.02374843  | test loss: 0.062073115
epoch: 60  | training loss: 0.016426945  | test loss: 0.0688743
epoch: 70  | training loss: 0.010725553  | test loss: 0.07921874
epoch: 80  | training loss: 0.007073321  | test loss: 0.0888902
epoch: 90  | training loss: 0.0046950025  | test loss: 0.10200834


In [89]:
with tf.Session() as sess:
    saver.restore(sess, 'breast_cancer.ckpt')
    pred = sess.run(y_hat, feed_dict={X: X_test})
    
classes = (pred > .5).astype(int)
metrics.accuracy_score(y_test, classes)

INFO:tensorflow:Restoring parameters from breast_cancer.ckpt


0.965034965034965

## Multiclass Classification

In [112]:
data = load_iris()
target = to_categorical(data.target)

X_train, X_test, y_train, y_test = train_test_split(data.data, target)

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [113]:
tf.reset_default_graph()

#None == allow dynamic number of rows; X_train.shape[1] == number of features
X = tf.placeholder(dtype=tf.float32, shape=(None, X_train.shape[1]), name='X')
y = tf.placeholder(dtype=tf.float32, shape=(None, 3), name='y') # shape has 3 for 3 classes

# first hidden layer
# X == features coming in, 4 == number of features == neurons
h1 = tf.layers.dense(X, 4, name='hidden1', activation=tf.nn.relu)

# last/output with 3 neurons == one for each class
y_hat = tf.layers.dense(h1, 3, name='y_hat', activation=None) #FYI: activation is None because softmax is taken care of in the loss function in tf

loss = tf.losses.softmax_cross_entropy(y, y_hat)

gd = tf.train.AdamOptimizer(.01) #could pass in learning rate as param
training_op = gd.minimize(loss)

saver = tf.train.Saver()

In [114]:
####### FYI: this/batching is required/good for large datasets only 
kf = StratifiedKFold()
batches = []
for train, test in kf.split(X_train, y_train.argmax(axis=1)):
#     print('train',train)
#     print('test',test)
    batches.append(test)

In [118]:
init = tf.global_variables_initializer()

###### WITHOUT BATCHING
# with tf.Session() as sess:
#     init.run()
#     #train model
#     for epoch in range(100):
#         sess.run(training_op, feed_dict={X: X_train, y: y_train})
#         if (epoch % 10 == 0):
#             training_loss = sess.run(loss, feed_dict={X: X_train, y: y_train})
#             test_loss = sess.run(loss, feed_dict={X: X_test, y: y_test})
#             print('epoch:',epoch,' | training loss:', training_loss, ' | test loss:', test_loss)
            
#     saver.save(sess, './iris.ckpt')
#     pred = sess.run(y_hat, feed_dict={X: X_test})

###### WITH BATCHING
with tf.Session() as sess:
    init.run()
    for epoch in range(100):
        for batch in batches:
            X_batch = X_train[batch]
            y_batch = y_train[batch]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 10 == 0:
            training_loss = sess.run(loss, feed_dict={X: X_train, y: y_train})
            test_loss = sess.run(loss, feed_dict={X: X_test, y: y_test})
            print('training loss', training_loss, 'test loss', test_loss)

    saver.save(sess, './iris.ckpt')       
    pred = sess.run(y_hat, feed_dict={X: X_test})

training loss 0.92804366 test loss 1.0773953
training loss 0.46016827 test loss 0.5540002
training loss 0.30851045 test loss 0.36973476
training loss 0.21520232 test loss 0.27755782
training loss 0.13728611 test loss 0.20214576
training loss 0.099481635 test loss 0.16531575
training loss 0.0791089 test loss 0.14572659
training loss 0.067743056 test loss 0.13391523
training loss 0.060723662 test loss 0.12670164
training loss 0.05573219 test loss 0.12180179


In [111]:
pred.argmax(axis=1)

array([1, 2, 0, 1, 2, 2, 0, 0, 0, 0, 1, 2, 0, 2, 2, 1, 2, 0, 0, 0, 2, 1,
       2, 0, 1, 2, 2, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2, 0])